In [21]:
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import *
from sklearn.metrics import roc_curve, auc

np.random.seed(0)
%matplotlib inline

X = np.genfromtxt('uci-cs178-f21/X_train.txt', delimiter=',')
Y = np.genfromtxt('uci-cs178-f21/Y_train.txt', delimiter=',')

X,Y = ml.shuffleData(X,Y)
# Fetureset1_X = X

# X = X[:,0:41]
Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)

Xte = np.genfromtxt('uci-cs178-f21/X_test.txt', delimiter=',')
# Xte = Xte[:,:41]

Xtr, param = ml.transforms.rescale(Xtr)
Xva, _ = ml.transforms.rescale(Xva,param)
Xte, _ = ml.transforms.rescale(Xte,param)

In [22]:
# print(X.shape)
# print(Y.shape)
# print(Xte.shape)

# Fetureset1_X = X[:,0:41]
# print(Fetureset1_X.shape)
# Fetureset2_X = X[:,41:69]
# print(Fetureset2_X.shape)
# Fetureset3_X = X[:,69:107]
# print(Fetureset3_X.shape)

# print(Fetureset1_X[0])
# print(Fetureset2_X[0])
# print(Fetureset3_X[0])

# # for i in X[:1]:
# #     for j in i :
# #         x += 1
# #         print(x,j)


(7423, 41)
(7423, 28)
(7423, 38)
[8.3300000e+02 0.0000000e+00 4.0000000e+00 6.4000000e+01 8.0000000e+01
 4.0000000e+00 1.6000000e+01 2.2008000e+02 0.0000000e+00 9.1096800e+05
 0.0000000e+00 2.2600000e+00 9.0000000e+00 6.0000000e+00 6.4000000e+01
 0.0000000e+00 1.2817760e+06 0.0000000e+00 8.8652090e+04 0.0000000e+00
 8.0000000e+00 2.4128687e+05 2.4000000e+01 3.2000000e+01 6.6016400e+04
 2.2000000e+02 2.4000000e+01 9.0000000e+02 4.0000000e+01 0.0000000e+00
 3.8154500e+05 1.5738200e+05 0.0000000e+00 0.0000000e+00 9.0000000e+00
 1.8123700e+05 2.2419600e+06 4.0000000e+00 5.6064000e+05 0.0000000e+00
 0.0000000e+00]
[1.290e+02 2.500e+01 2.200e+01 0.000e+00 1.780e+02 2.030e+03 1.292e+03
 2.553e+03 1.000e+00 9.600e+01 2.000e+00 7.000e+00 1.000e+01 1.000e+00
 5.000e+00 0.000e+00 3.300e+01 1.548e+03 8.980e+03 2.000e+00 1.200e+01
 3.630e+02 4.000e+00 3.936e+03 1.000e+00 1.800e+01 2.000e+00 8.000e+00]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 

In [3]:
nBag = 100
classifiers2 = [ None ] * nBag

mt,nt = Xte.shape

Ye = np.zeros((mt,2))
auc_list = []

for i in range(nBag):
    Xi,Yi = ml.bootstrapData(X,Y,mt) 
    classifiers2[i] = ml.dtree.treeClassify(Xi, Yi, maxDepth = 25 , minLeaf = 4, minParent = 512, nFeatures = 41)
    #Train a model on data Xi,Yi
    auc_list.append(classifiers2[i].auc(Xi,Yi))
    #Train a model on data Xi,Yi
    Ye += classifiers2[i].predictSoft(Xte)
learner = Ye / 25
print(np.mean(auc_list))

0.7669373572808575


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import gaussian_process
from sklearn import linear_model


In [17]:
max = 0
currk = 0

for mf in range(10,40):
    rf = RandomForestClassifier(n_estimators= 50, max_features=mf, bootstrap=True, max_depth=10)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
    print ("maxF = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)

maxF = 10
Training: 0.9566702462784236
Validation: 0.7166203488709584
maxF = 11
Training: 0.950950242261997
Validation: 0.7102894438962093
maxF = 12
Training: 0.9521285074947767
Validation: 0.7243698119231439
maxF = 13
Training: 0.9519215594173005
Validation: 0.7100173419631972
maxF = 14
Training: 0.9487751317393566
Validation: 0.7111746821849422
maxF = 15
Training: 0.9511300737876548
Validation: 0.708647762233703
maxF = 16
Training: 0.9501577355069688
Validation: 0.721724981134266
maxF = 17
Training: 0.9533638422905886
Validation: 0.7117315841411737
maxF = 18
Training: 0.9429041146247279
Validation: 0.7145578162187265
maxF = 19
Training: 0.9478195852982906
Validation: 0.7143963690718059
maxF = 20
Training: 0.9465637145364574
Validation: 0.7135546670923549
maxF = 21
Training: 0.9464345989047863
Validation: 0.7155337551517966
maxF = 22
Training: 0.9407663319077287
Validation: 0.71419319962849
maxF = 23
Training: 0.9406963280898544
Validation: 0.726065913972253
maxF = 24
Training: 0.9389

In [19]:
max = 0
currk = 0

for mf in range(10,40):
    rf = RandomForestClassifier(n_estimators=100, max_features=mf, bootstrap=True, max_depth=10)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
#     print ("maxF = " + str(mf))
#     print ("Training: " + str(roc_auc_tr))
#     print ("Validation: " + str(roc_auc_va))
    
print (max, mf)

0.7230437685029314 39


In [8]:
max = 0
currk = 0
depths=[10,20,30,40,50,60,70]
for d in depths:
    rf = RandomForestClassifier(n_estimators=100, max_features=39, bootstrap=True, max_depth=d)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = d
    
    print ("depth = " + str(d))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)

depth = 10
Training: 0.9478624725643466
Validation: 0.7187844662448482
depth = 20
Training: 1.0
Validation: 0.7115229726591978
depth = 30
Training: 0.9999998865416242
Validation: 0.7086559252916933
depth = 40
Training: 0.9999999999999999
Validation: 0.7048310791199862
depth = 50
Training: 0.9999999999999999
Validation: 0.7132816581528995
depth = 60
Training: 1.0
Validation: 0.7067203735415337
depth = 70
Training: 0.9999999999999999
Validation: 0.7116853268125617
0.7187844662448482 10


In [16]:
max = 0
currk = 0
nbag=[(50,28),(100,39)]
for b,mf in nbag:
    rf = RandomForestClassifier(n_estimators= 50, max_features= 39, bootstrap=True, max_depth= 10)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = d
    
    print ("nbag = " + str(b))
    print ("mf = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)

nbag = 50
mf = 28
Training: 0.940434239241735
Validation: 0.7129351816915308
nbag = 100
mf = 39
Training: 0.9439128730440485
Validation: 0.7176516151970743
0.7176516151970743 70
